In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=88194ae99652b63c84614c529b78f0e0476f0ae705deb23dcac1435707db1dcc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("JollibeeData").getOrCreate()

from google.colab import files
uploaded = files.upload()

jollibee_df = spark.read.csv("/content/jollibee.csv", header=True, inferSchema=True)

jollibee_df.show()

TypeError: 'NoneType' object is not subscriptable

RDD

In [ ]:
jlb_rdd = jollibee_df.rdd

In [ ]:
# flatMap each value in the city that contains 2 words
words_rdd = jlb_rdd.flatMap(lambda row: row['city'].split())

# Output
words_rdd.take(20)

['Bogo',
 'Danao,',
 'Cebu',
 'Bacolod',
 'Bacolod',
 'Bacolod',
 'Bacolod',
 'Bacolod',
 'Liloan',
 'Cebu',
 'Consolacion,',
 'Cebu',
 'Cebu',
 'Mandaue',
 'Cebu',
 'Mandaue',
 'Mandaue',
 'Cebu',
 'Cebu',
 'Cebu']

In [ ]:
# Map the dataset to certain column
selected_columns_rdd = jlb_rdd.map(lambda row: (row['city'], row['state'], row['franchiseNm']))

# Output
selected_columns_rdd.collect()

[('Bogo', 'Cebu', 'Bright Bee 2018 Foods Inc'),
 ('Danao, Cebu', 'Cebu', 'Knotsberry Foods Corporation'),
 ('Bacolod', 'Negros Occidental', 'Freemont Foods Corporation'),
 ('Bacolod', 'Negros Occidental', 'Neo Foods Corp.'),
 ('Bacolod', 'Negros Occidental', 'Neo Foods Corp.'),
 ('Bacolod', 'Negros Occidental', 'Freemont Foods Corporation'),
 ('Bacolod', 'Negros Occidental', 'Neo Foods Corp.'),
 ('Liloan', 'Cebu', 'Bright Bee 2018 Foods Inc'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Consolacion, Cebu', 'Cebu', 'Golden Lion Foods (Consolacion) Group'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Mandaue', 'Cebu', 'Topfood Casuntingan Corporation'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Mandaue', 'Cebu', 'Freemont Foods Corporation'),
 ('Mandaue', 'Cebu', 'Balete Foods Corporation'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Cebu', 'Cebu', 'Freemont Foods Corporation'),
 ('Cebu', 'Cebu', 'Freemon

In [ ]:
# Filter the value of state
jlb_metromanila = jlb_rdd.filter(lambda row: row['state'] == "NCR")

# Output
print("There are", jlb_metromanila.count(), "Jollibee stores within the NCR in the Philippines")

There are 120 Jollibee stores within the NCR in the Philippines


In [ ]:
# Map the value of franchiseNm
franchise_rdd = jlb_metromanila.map(lambda row: row['franchiseNm'])

# Map each value to a tuple
franchise_tuples = franchise_rdd.map(lambda value: (value, 1))

# Reduce by key to count occurrences of each value
franchise_counts = franchise_tuples.reduceByKey(lambda a, b: a + b)

# Find the most frequent value
most_frequent_value = franchise_counts.max(key=lambda x: x[1])

# Output
print(f"The '{most_frequent_value[0]}' has the most branches in the NCR with {most_frequent_value[1]} occurrences.")

The 'Jollibee Foods Corporation' has the most branches in the NCR with 30 occurrences.


In [ ]:
# Map the RDD to get the franchise name within the NCR
mapped_rdd = franchise_counts.map(lambda x: (x[1], x[0]))

# Sort the RDD by key in descending order
sorted_rdd = mapped_rdd.sortByKey(ascending=False)

# Map the sorted RDD to the final format (franchiseName, frequency)
final_rdd = sorted_rdd.map(lambda x: (x[1], x[0]))

# Output
final_rdd.take(15)

[('Jollibee Foods Corporation', 30),
 ('Haricot Corporation', 3),
 ('Berkshire  Corporation', 3),
 ('Berkshire Corporation', 3),
 ('South Foodworks Philippines Inc.', 1),
 ('Progresso Fastfood Inc.', 1),
 ('DEKAHAVEN FOODS CORPORATION', 1),
 ('Chickie-Sarap Foods Corporation', 1),
 ('Royal Betterliving Food Express Inc.', 1),
 ('Chanbee Food Corporation', 1),
 ('Eclectus Food Ventures Corporation', 1),
 ('Honeystar Corporation', 1),
 ('PROGATE FOODS INC.', 1),
 ('Prodynamic Inc.', 1),
 ('"Julie ""G"" Foods Corporation"', 1)]

In [ ]:
# Map the state and city
pair_rdd = jlb_rdd.map(lambda row: ((row['state'], row['city'])))

# Group the values by the key
grouped_rdd = pair_rdd.groupByKey()

# Output
for (state), city in grouped_rdd.collect():
    print(f"State: {state}, City: {list(city)}")

State: Cebu, City: ['Bogo', 'Danao, Cebu', 'Liloan', 'Cebu', 'Consolacion, Cebu', 'Cebu', 'Mandaue', 'Cebu', 'Mandaue', 'Mandaue', 'Cebu', 'Cebu', 'Cebu', 'Cebu', 'Mandaue', 'Cebu', 'Cebu', 'Cebu', 'Cebu', 'Lapu-Lapu, Philippines', 'Lapu-Lapu, Philippines', 'Cebu', 'Cebu', 'Talisay city', 'Minglanilla', 'Lapu-Lapu, Philippines', 'Minglanilla', 'Lapu-Lapu, Philippines', 'Lapu-Lapu, Philippines', 'Carcar']
State: Negros Occidental, City: ['Bacolod', 'Bacolod', 'Bacolod', 'Bacolod', 'Bacolod', 'Dumaguete']
State: Capiz, City: ['Roxas', 'Roxas']
State: Leyte, City: [' Ormoc Leyte', 'Palo, Leyte', 'Tacloban', 'Tacloban']
State: Iloilo, City: ['Iloilo City', 'Iloilo City', 'Iloilo City', 'Pavia', 'Iloilo City', 'Iloilo City']
State: Masbate, City: ['Masbate City']
State: Bohol, City: ['Tagbilaran', 'Tagbilaran', 'Panglao']
State: Antique, City: ['San Jose de Buenavista']
State: Negros Oriental, City: ['Tanjay', 'Dumaguete']
State: Sorsogon, City: ['Sorsogon City']
State: Albay, City: ['Legaz

Data  Frame

In [ ]:
# Filter to get the values within NCR
jlb_mm_df = jollibee_df.filter(jollibee_df.state == "NCR")

# Select columns
jlb_group_df = jlb_mm_df.select('storeID','alias', 'franchiseNm', 'X24hours', 'driveThru', 'delivery', 'deliveryCharge', 'pickup', 'breakfast')

# Output
jlb_group_df.show(10)

+-------+--------------------+--------------------+--------+---------+--------+--------------+------+---------+
|storeID|               alias|         franchiseNm|X24hours|driveThru|delivery|deliveryCharge|pickup|breakfast|
+-------+--------------------+--------------------+--------+---------+--------+--------------+------+---------+
|    324|       Plaza Central|South Foodworks P...|   false|    false|   false|             0|  true|    false|
|     52|    Alabang Junction|Progresso Fastfoo...|   false|     true|    true|             0|  true|    false|
|    221|Dasmarinas Centra...|Jollibee Foods Co...|   false|     true|    true|             0|  true|    false|
|    160|               Pilar|Jollibee Foods Co...|   false|    false|    true|             0|  true|    false|
|    468|    New SM Southmall|Jollibee Foods Co...|   false|    false|    true|             0|  true|    false|
|    184|Shopwise Sucat (S...|Jollibee Foods Co...|   false|     true|    true|             0|  true|   

In [ ]:
# Remove duplicates in the datarame
jlb_nodup_df = jlb_group_df.dropDuplicates()

# Filter to get branches that have deliveries
jlb_delivery_df = jlb_nodup_df.filter(jlb_nodup_df.delivery == 'true')

# Order dataframe by deliverycharge
delfee_jlb_group_df = jlb_delivery_df.orderBy('deliveryCharge', ascending=False)

# Output
delfee_jlb_group_df.show(10)

+-------+--------------------+--------------------+--------+---------+--------+--------------+------+---------+
|storeID|               alias|         franchiseNm|X24hours|driveThru|delivery|deliveryCharge|pickup|breakfast|
+-------+--------------------+--------------------+--------+---------+--------+--------------+------+---------+
|    303|  BGC Triangle Drive|Jollibee Foods Co...|   false|     true|    true|             5|  true|    false|
|    121|             Zabarte|Grand Oasis Selec...|   false|    false|    true|             0|  true|    false|
|     58| DMMA 1 Ever Gotesco|Alls Well Food Co...|   false|    false|    true|             0|  true|    false|
|    825| Shaw Blvd Wack Wack|FREUDIG FOOD CONC...|   false|     true|    true|             0|  true|     true|
|    136|San Francisco del...|Guru Nanik Food C...|   false|    false|    true|             0|  true|    false|
|    354|   Aglipay Poblacion|GOLD SUNRISE FOOD...|   false|    false|    true|             0|  true|   

In [ ]:
most_city_ncr = jlb_mm_df.groupBy('city').count()
most_city_ncr.orderBy('count', ascending=False).show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|         Quezon City|   29|
|              Manila|   19|
|               Pasig|   10|
|              Makati|   10|
|              Taguig|    8|
|           Las Pi�as|    7|
|           Para�aque|    7|
|         Mandaluyong|    6|
|            Caloocan|    5|
|               Pasay|    4|
|          Muntinlupa|    3|
|          Valenzuela|    3|
|            Marikina|    3|
|            San Juan|    2|
|         MANDALUYONG|    1|
|             Malabon|    1|
|Pateros, Metro Ma...|    1|
|             Navotas|    1|
+--------------------+-----+



In [ ]:
# Create a temporary table
jlb_mm_df.createOrReplaceTempView("jollibee_ncr")

# SQL query
most_city_ncr_sql = """   SELECT city, COUNT(*) as count
                          FROM jollibee_ncr
                          GROUP BY city
                          ORDER BY count DESC   """

# Execution and Output
most_city_ncr = spark.sql(most_city_ncr_sql)
most_city_ncr.show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|         Quezon City|   29|
|              Manila|   19|
|               Pasig|   10|
|              Makati|   10|
|              Taguig|    8|
|           Las Pi�as|    7|
|           Para�aque|    7|
|         Mandaluyong|    6|
|            Caloocan|    5|
|               Pasay|    4|
|            Marikina|    3|
|          Valenzuela|    3|
|          Muntinlupa|    3|
|            San Juan|    2|
|         MANDALUYONG|    1|
|             Malabon|    1|
|Pateros, Metro Ma...|    1|
|             Navotas|    1|
+--------------------+-----+



In [ ]:
# Create a temporary table
jlb_mm_df.createOrReplaceTempView("jollibee_ncr")

# SQL Query
jlb_24hrs_sql = """   SELECT *
                      FROM jollibee_ncr
                      WHERE X24hours == "true"
                """

# Execution and Output
jlb_24hrs_sql = spark.sql(jlb_24hrs_sql)
jlb_24hrs_sql.show()

+---+-------+---------+--------------------+---------+-----+---------+----------+--------------+-----------------+--------+---------+--------+---------+------+
|_c0|storeId|storeName|         franchiseNm|     city|state| latitude| longitude|deliveryCharge|            alias|X24hours|driveThru|delivery|breakfast|pickup|
+---+-------+---------+--------------------+---------+-----+---------+----------+--------------+-----------------+--------+---------+--------+---------+------+
|174|    511|   JB-246|Jollibee Foods Co...|Para�aque|  NCR|14.500274|120.996822|             0|Uni Oil Parañaque|    true|     true|    true|     true|  true|
+---+-------+---------+--------------------+---------+-----+---------+----------+--------------+-----------------+--------+---------+--------+---------+------+



In [ ]:
# Create a temporary table
jlb_mm_df.createOrReplaceTempView("jollibee_ncr")

# SQL Query
jlb_driveThru_sql = """   SELECT *
                      FROM jollibee_ncr
                      WHERE driveThru == "true"
                """

# Execution and Output
jlb_driveThru_sql = spark.sql(jlb_driveThru_sql).count()
print(jlb_driveThru_sql)

42


In [ ]:
# Group the DataFrame by 'city' and count the occurrences
city_counts = jlb_mm_df.groupBy('city').count()

# Order the DataFrame by count in descending order and get the first row (highest count)
mode_city = city_counts.orderBy('count', ascending=False).first()

# Print the mode city and its count
print(f"The city with the most Jollibee stores is {mode_city['city']} with {mode_city['count']} stores.")

The city with the most Jollibee stores is Quezon City with 29 stores.


In [ ]:
# Filter the DataFrame for branches with pickup, delivery, and drive-thru
filtered_df = jlb_mm_df.filter((jlb_mm_df.pickup == 'true') & (jlb_mm_df.delivery == 'true') & (jlb_mm_df.driveThru == 'true'))

# Group the filtered DataFrame by 'city' and count the occurrences
city_counts = filtered_df.groupBy('city').count()

# Display the result
city_counts.show()

+-----------+-----+
|       city|count|
+-----------+-----+
|Mandaluyong|    3|
|   Marikina|    1|
|      Pasay|    1|
|     Taguig|    2|
|      Pasig|    4|
|  Para�aque|    3|
|     Manila|    2|
|   San Juan|    1|
|Quezon City|   13|
| Muntinlupa|    2|
|  Las Pi�as|    2|
|   Caloocan|    1|
|     Makati|    2|
+-----------+-----+



In [ ]:
# Group the DataFrame by 'franchiseNm' and count the occurrences
franchise_counts = jlb_mm_df.groupBy('franchiseNm').count()

# Display the result
franchise_counts.show()

+--------------------+-----+
|         franchiseNm|count|
+--------------------+-----+
|"Eeboy ""G"" Food...|    1|
|Jolly Star Foods ...|    1|
|3Pals Food Corpor...|    1|
|Lucky Bee's Foods...|    1|
|GTWJ Food Corpora...|    1|
|Zenith Holdings C...|    1|
|   K88 Food Services|    1|
|Jollibee Foods Co...|   30|
|Veneto Foods Corp...|    1|
|Belarverde Foods ...|    1|
|Bongreat Foods Co...|    1|
|Guru Nanik Food C...|    1|
|Chickie-Sarap Foo...|    1|
| Castello Foods Inc.|    1|
|Hanspete Food Cor...|    1|
|Alls Well Food Co...|    1|
|Great Aduana Food...|    1|
|My Sheri Burgers ...|    1|
|           KNC Group|    1|
|OM Shanti Foods C...|    1|
+--------------------+-----+
only showing top 20 rows

